In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
energy_df = pd.read_csv('energydata_complete.csv')

In [4]:
energy_df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [5]:
energy_df.isnull().sum()

date           0
Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

In [36]:
#Fitting a regression model between T2 and T6

data_sample = energy_df[['T2','T6']]

In [37]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

In [38]:
Scaler = MinMaxScaler()

In [39]:
#Normalizing the two features

data_sample_norm = pd.DataFrame(Scaler.fit_transform(data_sample), columns=['T2', 'T6'])

In [40]:
data_sample_norm

,T2,T6
0,0.225345,0.381070
1,0.225345,0.375443
2,0.225345,0.367487
3,0.225345,0.363800
4,0.225345,0.361859
...,...,...
19730,0.711655,0.898317
19731,0.701769,0.880852
19732,0.692651,0.864260
19733,0.677054,0.829525


In [41]:
X = np.array(data_sample_norm['T2'])
y = np.array(data_sample_norm['T6'])

In [42]:
X = X.reshape(-1, 1)
y = y.reshape(-1, 1)

In [43]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=42)

In [45]:
lin_reg = LinearRegression()
lin_reg.fit(train_X, train_y)

LinearRegression()

In [46]:
y_pred = lin_reg.predict(test_X)

In [47]:
R2_score = round(r2_score(test_y, y_pred), 2)
R2_score

#R2_score = 0.64

0.64

In [52]:
energy_df = energy_df.drop(columns = ['date', 'lights'])

In [53]:
#Running a Multiple Linear regression on the Test data

energy_df_norm = pd.DataFrame(Scaler.fit_transform(energy_df), columns=energy_df.columns)

In [55]:
X = np.array(energy_df_norm.drop(['Appliances'], axis = 1))
y = np.array(energy_df_norm['Appliances'])

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [57]:
linear_reg =LinearRegression()
linear_reg.fit(X_train, y_train)

LinearRegression()

In [60]:
prediction = linear_reg.predict(X_test)

In [61]:
MAE  = round(mean_absolute_error(y_test, prediction), 2)
MAE

#MAE is 0.05

0.05

In [62]:
RSS = round(np.sum(np.square(y_test - prediction)), 2)

RSS

#RSS is 43.35

45.35

In [63]:
RMSE = round(np.sqrt(mean_squared_error(y_test, prediction)), 3)

RMSE


#RMSE is 0.088

0.088

In [64]:
R2_score = round(r2_score(y_test, prediction), 2)

R2_score

#Coefficeint  of Determination is 0.15


0.15

In [65]:
lin_reg_weights = pd.DataFrame(np.transpose(linear_reg.coef_), energy_df.drop(['Appliances'], 1).columns, columns=['Weights'])

lin_reg_weights

C:\Users\HP\AppData\Local\Temp/ipykernel_11188/1435497868.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  lin_reg_weights = pd.DataFrame(np.transpose(linear_reg.coef_), energy_df.drop(['Appliances'], 1).columns, columns=['Weights'])


,Weights
T1,-0.003281
RH_1,0.553547
T2,-0.236178
RH_2,-0.456698
T3,0.290627
RH_3,0.096048
T4,0.028981
RH_4,0.026386
T5,-0.015657
RH_5,0.016006


In [66]:
lin_reg_weights.sort_values('Weights')


#Highest is RH1, Lowest is RH2

,Weights
RH_2,-0.456698
T_out,-0.321860
T2,-0.236178
T9,-0.189941
RH_8,-0.157595
RH_out,-0.077671
RH_7,-0.044614
RH_9,-0.039800
T5,-0.015657
T1,-0.003281


In [67]:
from sklearn.linear_model import Ridge, Lasso

In [68]:
ridge = Ridge(alpha = 0.4)
ridge.fit(X_train, y_train)

pred = ridge.predict(X_test)

ridge_rmse = round(np.sqrt(mean_squared_error(y_test, pred)), 3)

ridge_rmse


#Initial RMSE was 0.088 as well, hence no changes.


0.088

In [73]:
lasso = Lasso(alpha = 0.001)
lasso.fit(X_train, y_train)

pred_lasso = lasso.predict(X_test)

In [74]:
lasso_weights_df = pd.DataFrame(np.transpose(lasso.coef_), energy_df.drop(['Appliances'], 1).columns, columns=['Lasso_weight'])

C:\Users\HP\AppData\Local\Temp/ipykernel_11188/2811578287.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  lasso_weights_df = pd.DataFrame(np.transpose(lasso.coef_), energy_df.drop(['Appliances'], 1).columns, columns=['Lasso_weight'])


In [75]:
non_zero_weights = lasso_weights_df[lasso_weights_df['Lasso_weight'] != 0]

non_zero_weights


#Hence we have 4 features with non_zero feature weights

,Lasso_weight
RH_1,0.017880
RH_8,-0.000110
RH_out,-0.049557
Windspeed,0.002912


In [76]:
lasso_RMSE = round(np.sqrt(mean_squared_error(y_test, pred_lasso)), 3)

lasso_RMSE

#RMSE with lasso regression is 0.094

0.094